In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract
! sudo apt-get install tesseract-ocr-script-hang tesseract-ocr-script-hang-vert

In [ ]:
#OpenCV - 문제 크롭하기 (미완) 


#라이브러리 임포트
import numpy as np
import cv2 #openCV package
from google.colab.patches import cv2_imshow
import re
import pytesseract
import glob
import os
import time
from PIL import ImageEnhance, ImageFilter, Image
from pytesseract import Output


#수능완성 page trim 
#페이지에서 왼쪽,오른쪽 반페이지 분리 
#가진 문제집 데이터에 따라 별도의 값 입력이 필요함 
def imtrim(page):
    x=300
    w=1120
    left = page[465:3450, x:x+w]
    x=1480
    right = page[465:3450,x:x+w]

    return right,left
    
    



#반페이지를 입력받고 크롭하기 
def contour(page_rl):

    #이미지 흑백화 
    imgray = cv2.cvtColor(page_rl, cv2.COLOR_BGR2GRAY) 

    #이미지 이진화 (스캔본 처럼)
    blur = cv2.GaussianBlur(imgray, ksize=(3,3), sigmaX=0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]


    # Morph operations
    edge = cv2.Canny(imgray, 100, 200)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1000,100))
    closed = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel)

	
    contours, hierarchy = cv2.findContours(closed.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


    contours_xy = np.array(contours,dtype=object)
    contours_xy.shape

	#한페이지 내의 모든 폐곡선 범위에 대해 실행 
    for c in contours:

		#폐곡선 바운더리 
        x,y,w,h = cv2.boundingRect(c)

		#바운더리를 이용해서 문제 크롭 
        img_trim = page_rl[y:y+h , x:x+w]
        include(img_trim)


def include(cropped):
    save_path='/content/drive/MyDrive/cropped/suwan_ga/'
    d = pytesseract.image_to_string(cropped, lang='eng+Hangul')
    if(d[0].isdigit()):
        new_name=save_path+page+'_'+d[0:3].replace(' ','')+'.png' #input image type
        print(new_name)
        cv2.imwrite(new_name,cropped)
        

    
    
#페이지 url을 입력하면 
def problem_crop(image_url):

    #이미지url을 open CV에 읽어들이기 
    imgfile = image_url
    image = cv2.imread(imgfile)

#한 페이지를 반페이지로 나누기 
    right,left = imtrim(image)
    
#각 반페이지에 대해 크롭 진행 
    contour(right)
    contour(left)
 

#함수실행 
if __name__ == "__main__":
    
    #크롭을 진행할 폴더의 경로를 입력 
    os.chdir('input_your_path')
    images=glob.glob('./*.png') #input image type(png/jpg)

    error_list=[]
    for i in range(len(images)):
        filename=images[i]
        page=filename[11:14] #사용자가 page를 파일이름에 넣고 싶을 시에만 사용 (범위는 파일에따라 다르게지정 필요)
        problem_crop(filename)
